# Airbnb Property Price Finder

Data Analysts:
    Justin Frank,
    Leo Ramirez,
    Tari Okoya-Koren,
    Veohnti Afokpa,
    Araz Ohanessian,
    Milton Dimas

In [2]:
import ipywidgets as widgets
from ipywidgets import Layout, Button, Box

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os

## With the data now cleaned and ready for use, it is imported for use in input selection

In [4]:
filepath = os.path.join('../df.csv')

In [5]:
df = pd.read_csv(filepath)
len(df.columns)
df.head()

c:\users\justin\appdata\local\programs\python\python38-32\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (50) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_cleansed,property_type,room_type,...,child_friendly,parking,host_greeting,internet,long_term_stays,pets_allowed,private_entrance,self_check_in,time_since_first_review,time_since_last_review
0,2060,2008-08-18,a few days or more,0-49%,0.0,0.0,0.0,Washington Heights,Other,Private room,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,4+ years,3851.0
1,2595,2008-09-09,within a few hours,50-89%,0.0,6.0,1.0,Midtown,Apartment,Entire home/apt,...,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,4+ years,-209.0
2,3831,2008-12-07,within an hour,90-99%,0.0,1.0,1.0,Clinton Hill,Other,Entire home/apt,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,4+ years,-425.0
3,5099,2009-02-02,unknown,unknown,0.0,1.0,0.0,Murray Hill,Apartment,Entire home/apt,...,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,4+ years,-187.0
4,5121,2009-02-03,unknown,unknown,0.0,1.0,0.0,Bedford-Stuyvesant,Apartment,Private room,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,4+ years,-237.0


In [6]:
df.columns.tolist()

['id',
 'host_since',
 'host_response_time',
 'host_response_rate',
 'host_is_superhost',
 'host_listings_count',
 'host_identity_verified',
 'neighbourhood_cleansed',
 'property_type',
 'room_type',
 'accommodates',
 'bathrooms',
 'bedrooms',
 'beds',
 'price',
 'security_deposit',
 'cleaning_fee',
 'guests_included',
 'extra_people',
 'minimum_nights',
 'maximum_nights',
 'availability_90',
 'number_of_reviews',
 'first_review',
 'review_scores_rating',
 'review_scores_accuracy',
 'review_scores_cleanliness',
 'review_scores_checkin',
 'review_scores_communication',
 'review_scores_location',
 'review_scores_value',
 'instant_bookable',
 'cancellation_policy',
 'host_days_active',
 'air_conditioning',
 'bed_linen',
 'tv',
 'coffee_machine',
 'cooking_basics',
 'white_goods',
 'elevator',
 'gym',
 'child_friendly',
 'parking',
 'host_greeting',
 'internet',
 'long_term_stays',
 'pets_allowed',
 'private_entrance',
 'self_check_in',
 'time_since_first_review',
 'time_since_last_review'

## By creating lists of the unique items within a given feature, we can create dropdowns and consistent selectables for inquiring future hosts

In [7]:
def listMaker(column):
    '''
    Creates a list of unique items from a given column.
    Input column name in string format.
    '''
    new_list = list(df[column].unique())
    return new_list

In [8]:
max(listMaker('bathrooms'))

15.5

In [9]:
style = {'description_width': 'initial'}

In [10]:
room_types = listMaker('room_type')
room_type = widgets.Select(
    options=room_types,
    value=room_types[0],
    description='Property Type:',
    disabled=False
)

In [11]:
neighborhoods = listMaker('neighbourhood_cleansed')
neighborhood = widgets.Dropdown(
    options=neighborhoods,
    value=neighborhoods[0],
    description='Neighborhood:',
    disabled=False,
    style=style
)

In [29]:
bathroom_count = widgets.BoundedFloatText(
    value=2.5,
    step=0.5,
    description='Bathrooms:',
    disabled=False,
    style=style
)
bathroom_count

BoundedFloatText(value=2.5, description='Bathrooms:', step=0.5, style=DescriptionStyle(description_width='init…

In [13]:
prop_types = listMaker('property_type')
property_type = widgets.Select(
    options=prop_types,
    value='Other',
    # rows=10,
    description='Property Type:',
    disabled=False
)

In [80]:
new_strings=[]
for amen in amenities:
    new_amen = amen.replace('_', ' ')
    new_amen = new_amen[0].upper()+new_amen[1:]
    new_strings.append(new_amen)
    
amenities = df.columns[34:49].tolist()
items_amenities = [widgets.Checkbox(
    value=False,
    description=string,
    disabled=False
) for string in new_strings]
amen_checklist = widgets.GridBox(items_amenities, 
                                 layout=Layout(
                                        width='80%',
                                        grid_template_rows='auto auto auto auto',
                                        grid_template_columns='20% 20% 20% 20%',
                                        grid_template_areas='''
                                        "header header header header"
                                        "main main . sidebar "
                                        "footer footer footer footer"
                                        '''))
amen_checklist

GridBox(children=(Checkbox(value=False, description='Air conditioning'), Checkbox(value=False, description='Be…

In [81]:
amen_values = [amen_checklist.children[i].value for i in range(0, len(amen_checklist.children))]
amen_values

[False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [84]:
amen_tuples = zip(amenities, amen_values)
list(amen_tuples)

[('air_conditioning', False),
 ('bed_linen', False),
 ('tv', False),
 ('coffee_machine', False),
 ('cooking_basics', False),
 ('white_goods', False),
 ('elevator', False),
 ('gym', False),
 ('child_friendly', False),
 ('parking', False),
 ('host_greeting', False),
 ('internet', False),
 ('long_term_stays', False),
 ('pets_allowed', False),
 ('private_entrance', False)]

In [17]:
accommodation_count = widgets.BoundedIntText(
    value=3,
    step=1,
    description='Accommodates:',
    disabled=False,
    style=style
)

In [40]:
layout_fees = Layout( width='85%')
cleaning_fee = widgets.RadioButtons(
    options=['Yes', 'No'],
    description='Cleaning Fee?',
    disabled=False,
    style=style
)
security_deposit = widgets.RadioButtons(
    options=['Yes', 'No'],
    description='Security Deposit?',
    disabled=False,
    style=style
)
guests_included = widgets.IntSlider(
    value=1,
    min=1,
    max=16,
    step=1,
    description='Guests Included:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d',
    style=style,
    layout=layout_fees
)
extra_ppl_cost = widgets.RadioButtons(
    options=['Yes', 'No'],
    description='Extra Persons Fee?',
    disabled=False,
    style=style
)

In [53]:
minimum_nights = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1500,
    step=1,
    description='Minimum Night Stay:',
    disabled=False,
    style=style
)
maximum_nights = widgets.BoundedIntText(
    value=1,
    min=1,
    max=1500,
    step=1,
    description='Maximum Nights:',
    disabled=False,
    style=style
)
availability_90 = widgets.BoundedIntText(
    value=1,
    min=0,
    max=90,
    step=1,
    description='Available Days out of 90:',
    disabled=False,
    style=style
)

In [41]:
fees_vbox = widgets.VBox([cleaning_fee, extra_ppl_cost, security_deposit])

In [47]:
accordion_property = widgets.Accordion(children=[neighborhood, property_type, room_type, bathroom_count, accommodation_count])
accordion_property.set_title(0, 'Neighborhood')
accordion_property.set_title(1, 'Property Type')
accordion_property.set_title(2, 'Room Type')
accordion_property.set_title(3, 'Bathrooms')
accordion_property.set_title(4, 'Total Accommodation')
accordion_property.selected_index = None
accordion_property

Accordion(children=(Dropdown(description='Neighborhood:', options=('Washington Heights', 'Midtown', 'Clinton H…

In [55]:
accordion_dimensions = widgets.Accordion(children=[availability_90, minimum_nights, maximum_nights, fees_vbox])
accordion_dimensions.set_title(0, 'Availability 90')
accordion_dimensions.set_title(1, 'Minimum Nights')
accordion_dimensions.set_title(2, 'Maximum Nights')
accordion_dimensions.set_title(3, 'Optional Fees')
accordion_dimensions.selected_index = None
accordion_dimensions

Accordion(children=(BoundedIntText(value=1, description='Available Days out of 90:', max=90, style=Description…

In [85]:
UserInput = widgets.Tab()
UserInput.children = [accordion_property, accordion_dimensions, amen_checklist]
UserInput.set_title(0, 'Property Details')
UserInput.set_title(1, 'Availability and Fees')
UserInput.set_title(2, 'Amenities Offered')

## Using the features determined to hold the most weight in determining price, we built an input form regarding a prospective host's property dimensions

In [86]:
UserInput

### Click to summarize your property information

In [ ]:
def PropertyDetails(b):
    with output:
        prop_details = f'''
Review of Property Details

    Borough:               {borough.value} 
    Neighborhood:          {neighborhood.value}
    Zip:                   {zipcode.value}

    Property Type:         {property_type.value}
    Accommodation Count:   {accommodation_count.value}
    Bedroom Count:         {bedroom_count.value} 
    Bathroom Count:        {bath_count.value}
    Bed Count:             {bed_count.value} 
    Bed Type:              {bed_type.value}''')
        

In [ ]:
from IPython.display import display
button = widgets.Button(description="Click Me!", button_style='success')
output = widgets.Output()

display(button, output)
button.on_click(PropertyDetails)